**DATA 2060 Final Notebook**

blah blah

In [ ]:
import numpy as np

def softmax(x):
    '''
    Apply softmax to an array
    @params:
        x: the original array
    @return:
        an array with softmax applied elementwise.
    '''
    inner = np.array(x - np.max(x))
    e = np.exp(inner.astype(np.float64))
    return (e + 1e-6) / (np.sum(e) + 1e-6)

class LogisticRegression:
    '''
    Multiclass Logistic Regression that learns weights using 
    stochastic gradient descent.
    '''
    def __init__(self, n_features, n_classes, batch_size, conv_threshold):
        '''
        Initializes a LogisticRegression classifer.
        @attrs:
            n_features: the number of features in the classification problem
            n_classes: the number of classes in the classification problem
            weights: The weights of the Logistic Regression model
            alpha: The learning rate used in stochastic gradient descent
        '''
        self.n_classes = n_classes
        self.n_features = n_features
        self.weights = np.zeros((n_classes, n_features + 1))  # An extra row added for the bias
        self.alpha = 0.03  # DO NOT TUNE THIS PARAMETER
        self.batch_size = batch_size
        self.conv_threshold = conv_threshold

    def train(self, X, Y):
        '''
        Trains the model using stochastic gradient descent
        @params:
            X: a 2D Numpy array where each row contains an example, padded by 1 column for the bias
            Y: a 1D Numpy array containing the corresponding labels for each example
        @return:
            num_epochs: integer representing the number of epochs taken to reach convergence
        '''

        converge = False
        num_epochs = 0
        n = len(X)
        b = self.batch_size
        while not converge:
            num_epochs += 1

            # shuffle training examples using indicies
            p = np.random.permutation(n)
            X_ = X[p]
            Y_ = Y[p]

            # calculate last epoch loss
            e_last = self.loss(X_, Y_)

            # iterate through each batch, updating weights for each batch
            for i in range(int(np.ceil(n/b))):
                # get examples and labels in current batch
                x_batch = X_[i*b : (i+1)*b]
                y_batch = Y_[i*b : (i+1)*b]
                n_batch = len(x_batch)

                # initialize Lw as a matrix of 0s
                Lw = np.zeros((self.n_classes, self.n_features + 1))  # An extra row added for the bias

                # for each data point in batch
                for d in range(n_batch):
                    x = x_batch[d]
                    y = y_batch[d]

                    probabilities = self.getSigmoidProbability(x)

                    for j in range(self.n_classes):
                        # calculate partial derivative of loss with respect to single row in weights matrix
                        h_w_x = probabilities[j] # probabilitiy at j

                        Lw[j,:] = Lw[j,:] + (h_w_x-1) * x
                    else:
                        Lw[j,:] = Lw[j,:] + h_w_x * x

                # update weights
                self.weights -= (self.alpha * Lw / n_batch)

            # calculate current epoch loss
            e_current = self.loss(X_,Y_)

            # check convergence
            if np.abs(e_current - e_last) < self.conv_threshold:
                converge = True

        print("Finished training with ", num_epochs, "epochs.")
        return num_epochs

    def loss(self, X, Y):
        '''
        Returns the total log loss on some dataset (X, Y), divided by the number of examples.
        @params:
            X: 2D Numpy array where each row contains an example, padded by 1 column for the bias
            Y: 1D Numpy array containing the corresponding labels for each example
        @return:
            A float number which is the average loss of the model on the dataset
        '''   
        # calculate loss for each example (x,y), then average them
            
        total_loss = 0
        n = len(Y)
        predictions = self.predict(X)

        for i in range(n):
            loss = 0
            probabilities = self.getSoftmaxProbability(X[i])

            for j in range(self.n_classes):
                if Y[i] == j: 
                    # calculate our probability that x in j based on weights:
                    h_w_x = probabilities[j]
                    if h_w_x > 0: loss += np.log(h_w_x)
            total_loss += loss

        # return avg loss
        avg_loss = -total_loss/n
        return avg_loss


    def getSigmoidProbability(self, x):
        '''
        Compute softmax sigmoid probability vector of an example x belonging to each possible class.
        @params:
            x: a 1D Numpy array that is one example from the training set, padded by 1 column for the bias
        @return:
            A 1D Numpy array with one element for each possible class j, 
            where the element at j represents the probability of x belonging to j.
        '''

        probs = []
        for j in range(self.n_classes):
            # binary logistical predictor to see if a given x is in j, using the weights of class j
            # outputs continuous float from [0,1]
            # do this for each possible class, then pick the one with highest probablity (softmax) as the verdict class for this x
            h_w_x = 1/(1+ np.exp(-self.weights[j].dot(x))) 
            probs.append(h_w_x)
        
        normalized_prob = softmax(probs)
        return normalized_prob
        
    def getSoftmaxProbability(self, x):
        '''
        Compute softmax regular probability vector of an example belonging to each possible class.
        @params:
            x: a 1D Numpy array that is one example from the training set, padded by 1 column for the bias
        @return:
            A 1D Numpy array with one element for each possible class j, 
            where the element at j represents the probability of x belonging to j.
        '''
        return softmax(np.dot(self.weights, x))

    def predict(self, X):
        '''
        Compute predictions based on the learned weigths and examples X
        @params:
            X: a 2D Numpy array where each row contains an example, padded by 1 column for the bias
        @return:
            A 1D Numpy array with one element for each row in X containing the predicted class.
        '''
        # use one vs all algorithm for returning class with highest probablity of that x belonging to it
        # let j be each possible class

        predictions = []
        for x in X:
            normalized_prob = self.getSoftmaxProbability(x)
            # print("norm prob:", normalized_prob.shape)
            # the index of highest probablity is used as the predicted class
            x_prediction = np.argmax(normalized_prob)
            predictions.append(x_prediction)

        return np.array(predictions)

    def accuracy(self, X, Y):
        '''
        Outputs the accuracy of the trained model on a given testing dataset X and labels Y.
        @params:
            X: a 2D Numpy array where each row contains an example, padded by 1 column for the bias
            Y: a 1D Numpy array containing the corresponding labels for each example
        @return:
            a float number indicating accuracy (between 0 and 1)
        '''

        # count how many 0s (accurately predicted examples) from the difference
        predict = self.predict(X)
        diff = predict - Y
        num_correct = np.sum(diff == 0)

        accuracy = num_correct/len(Y)
        
        return accuracy

In [2]:
### One vs. All Logistic Regression multi-class regression classifier ###
import numpy as np
import itertools as it

class OneVsAll:
    def __init__(
        self,
        n_features: int,
        n_classes: int,
        batch_size: int,
        conv_threshold: float
    ) -> None:
        self.n_classes = n_classes
        self.models = [
            LogisticRegression(n_features, 2, batch_size, conv_threshold) 
            for _ in range(n_classes)
        ]

    def train(self, X: np.ndarray, Y: np.ndarray) -> None:
        # Train a binary classifier for each class against all others
        for class_label in range(self.n_classes):
            binary_labels = (Y == class_label).astype(int)
            self.models[class_label].train(X, binary_labels)

    def predict(self, X: np.ndarray) -> np.ndarray:
        # Get the probabilities of each class for each data point
        # print("X", X.shape)
        probabilities = np.array([
            model.predict(X) for model in self.models
        ])

        # Return the class with the highest probability for each data point
        return np.argmax(probabilities, axis=0)

    def accuracy(self, X: np.ndarray, Y: np.ndarray) -> float:
        predictions = self.predict(X)
        return np.mean(predictions == Y)


In [3]:
### All Pairs Logistic Regression multi-class regression classifier ###

class AllPairs:
    def __init__(
        self,
        n_features: int,
        n_classes: int,
        batch_size: int,
        conv_threshold: float,
    ):
        self.n_classes = n_classes
        self.pairs = list(it.combinations(range(n_classes), 2))
        self.models = {
            (i, j): LogisticRegression(
                n_features,
                2,
                batch_size,
                conv_threshold
            )
            for i, j in self.pairs
        }

    def train(self, X: np.ndarray, Y: np.ndarray) -> None:
        # Iterate over all pair combinations
        print("self pairs:", self.pairs)
        for i, j in self.pairs:
            print("ij", i,j)
            indices = (Y == i) | (Y == j)

            # Get appropriate data
            X_pair, Y_pair = X[indices], Y[indices]
            Y_pair = (Y_pair == i).astype(int)

            # Train on the pair
            self.models[(i, j)].train(X_pair, Y_pair)

    def predict(self, X: np.ndarray) -> np.ndarray:
        # Count predictions for each class
        votes = np.zeros((len(X), self.n_classes))
        for (i, j), model in self.models.items():
            predictions = np.array(model.predict(X))
            # print("predictions shape:", predictions.shape, predictions)
            votes[:, i] += predictions
            votes[:, j] += (1 - predictions)

        # Return the class with the most predictions
        return np.argmax(votes, axis=1)

    def accuracy(self, X: np.ndarray, Y: np.ndarray) -> float:
        predictions = self.predict(X)
        return np.mean(predictions == Y)


In [4]:
### Dummy test ###
n_samples = 500
n_features = 10
n_classes = 5
X = np.random.rand(n_samples, n_features + 1)
Y = np.random.randint(0, n_classes, n_samples)

#One-vs-All
ova = OneVsAll(n_features, n_classes, batch_size=32, conv_threshold=1e-4)
ova.train(X, Y)
print(f"One-vs-All Accuracy: {ova.accuracy(X, Y)}")

# All-Pairs
all_pairs = AllPairs(n_features, n_classes, batch_size=32, conv_threshold=1e-4)
all_pairs.train(X, Y)
print(f"All-Pairs Accuracy: {all_pairs.accuracy(X, Y)}")

# TODO: Add unit tests


Finished training with  61 epochs.
Finished training with  56 epochs.
Finished training with  71 epochs.
Finished training with  49 epochs.
Finished training with  49 epochs.
One-vs-All Accuracy: 0.164
self pairs: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
ij 0 1
Finished training with  136 epochs.
ij 0 2
Finished training with  159 epochs.
ij 0 3
Finished training with  161 epochs.
ij 0 4
Finished training with  160 epochs.
ij 1 2
Finished training with  126 epochs.
ij 1 3
Finished training with  111 epochs.
ij 1 4
Finished training with  111 epochs.
ij 2 3
Finished training with  189 epochs.
ij 2 4
Finished training with  187 epochs.
ij 3 4
Finished training with  111 epochs.
All-Pairs Accuracy: 0.194


## German Numerical Credit Data

In [5]:
import pandas as pd

def get_credit():
    """
    Gets and preprocesses German Credit data
    """
    data = pd.read_csv('./data/german_numerical-binsensitive.csv') # Reads file - may change

    # MONTH categorizing
    data['month'] = pd.cut(data['month'],3, labels=['month_1', 'month_2', 'month_3'], retbins=True)[0]
    # month bins: [ 3.932     , 26.66666667, 49.33333333, 72.        ]
    a = pd.get_dummies(data['month'])
    data = pd.concat([data, a], axis = 1)
    data = data.drop(['month'], axis=1)

    # CREDIT categorizing
    data['credit_amount'] = pd.cut(data['credit_amount'], 3, labels=['cred_amt_1', 'cred_amt_2', 'cred_amt_3'], retbins=True)[0]
    # credit bins: [  231.826,  6308.   , 12366.   , 18424.   ]
    a = pd.get_dummies(data['credit_amount'])
    data = pd.concat([data, a], axis = 1)
    data = data.drop(['credit_amount'], axis=1)

    for header in ['investment_as_income_percentage', 'residence_since', 'number_of_credits']:
        a = pd.get_dummies(data[header], prefix=header)
        data = pd.concat([data, a], axis = 1)
        data = data.drop([header], axis=1)

    # change from 1-2 classes to 0-1 classes
    data['people_liable_for'] = data['people_liable_for'] -1
    data['credit'] = -1*(data['credit']) + 2 # original encoding 1: good, 2: bad. we switch to 1: good, 0: bad

    # balance dataset
    data = data.reindex(np.random.permutation(data.index)) # shuffle
    pos = data.loc[data['credit'] == 1]
    neg = data.loc[data['credit'] == 0][:350]
    combined = pd.concat([pos, neg])

    y = data.iloc[:, data.columns == 'credit'].to_numpy()
    x = data.drop(['credit', 'sex', 'age', 'sex-age'], axis=1).to_numpy()

    # split into train and validation
    X_train, X_val, y_train, y_val = x[:350, :], x[351:526, :], y[:350, :].reshape([350,]), y[351:526, :].reshape([175,])

    # keep info about sex and age of validation rows for fairness portion
    x_sex = data.iloc[:, data.columns == 'sex'].to_numpy()[351:526].reshape([175,])
    x_age = data.iloc[:, data.columns == 'age'].to_numpy()[351:526].reshape([175,])
    x_sex_age = data.iloc[:, data.columns == 'sex-age'].to_numpy()[351:526].reshape([175,])

    return X_train, X_val, y_train, y_val, x_sex, x_age, x_sex_age

np.random.seed(0)
X_train, X_test, y_train, y_test, x_sex, x_age, x_sex_age = get_credit()
n_samples = len(X_train)
n_features = len(X_train[0])
n_classes = 2

# process data
X_train[X_train == False] = 0.0
X_train[X_train == True] = 1.0
X_test[X_test == False] = 0.0
X_test[X_test == True] = 1.0

train_bias = np.ones((n_samples, 1))
X_train = np.hstack((X_train, train_bias))

test_bias = np.ones((len(X_test), 1))
X_test = np.hstack((X_test, test_bias))

print("x shape:", X_train.shape, X_test.shape)
print("num features:", n_features)

batch_size = 50
conv_threshold = 1e-5

# One-vs-All
ova = OneVsAll(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold)
ova.train(X_train, y_train)
print(f"One-vs-All Accuracy: {ova.accuracy(X_test, y_test)}")

# All-Pairs

all_pairs = AllPairs(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold)
all_pairs.train(X_train, y_train)
print(f"All-Pairs Accuracy: {all_pairs.accuracy(X_test, y_test)}")


x shape: (350, 70) (175, 70)
num features: 69
Finished training with  35 epochs.
Finished training with  37 epochs.
One-vs-All Accuracy: 0.2914285714285714
self pairs: [(0, 1)]
ij 0 1
Finished training with  35 epochs.
All-Pairs Accuracy: 0.7085714285714285


## SkLearn Iris Data

In [6]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
Y = data.target

n_samples = len(X)
n_features = len(X[0])
n_classes = 3
batch_size = 10
conv_threshold = 1e-6

print("Number samples: ", n_samples, "Number Feats: ", n_features)

# shuffle data
p = np.random.permutation(n_samples)
X_ = X[p]
Y_ = Y[p]

# add bias to shuffled data
bias = np.ones((n_samples,1))
X_ = np.hstack((X_, bias))

# split data 80/20
train_size = int(.8*n_samples)
X_train = X_[:train_size,:]
y_train = Y_[:train_size]

X_test = X_[train_size:, :]
y_test = Y_[train_size:]


# One-vs-All
ova = OneVsAll(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold)
ova.train(X_train, y_train)
print(f"One-vs-All Accuracy: {ova.accuracy(X_test, y_test)}")

# All-Pairs

all_pairs = AllPairs(n_features, n_classes, batch_size=batch_size, conv_threshold=conv_threshold)
all_pairs.train(X_train, y_train)
print(f"All-Pairs Accuracy: {all_pairs.accuracy(X_test, y_test)}")


Number samples:  150 Number Feats:  4
Finished training with  5 epochs.
Finished training with  5 epochs.
Finished training with  4 epochs.
One-vs-All Accuracy: 0.43333333333333335
self pairs: [(0, 1), (0, 2), (1, 2)]
ij 0 1
Finished training with  7 epochs.
ij 0 2
Finished training with  8 epochs.
ij 1 2
Finished training with  5 epochs.
All-Pairs Accuracy: 0.4
